In [1]:
import numpy as np
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras import Model

In [2]:
# Data Importing
x_train = np.load('/content/drive/My Drive/Compition/xs.npy')
y_train = np.load('/content/drive/My Drive/Compition/ys.npy')

In [9]:
def build_model():
  vgg16 = VGG16(include_top = False, input_shape=(32,32,3),weights='imagenet')
  for layer in vgg16.layers:
    layer.trainable = False
  x = Flatten()(vgg16.output)
  x = Dense(500,activation='relu')(x)
  x = BatchNormalization()(x)
  x = Dropout(0.3)(x)
  x = Dense(250,activation='relu')(x)
  x = BatchNormalization()(x)
  x = Dropout(0.25)(x)
  x = Dense(70,activation='relu')(x)
  x = BatchNormalization()(x)
  x = Dropout(0.2)(x)
  x = Dense(9,activation='softmax')(x)
  m = Model(inputs = vgg16.inputs, outputs = x)
  return m

In [10]:
model = build_model()

In [11]:
early_stopping = EarlyStopping(monitor = 'val_accuracy', patience =  20, restore_best_weights = True)
lr_sechudle = ReduceLROnPlateau(monitor = 'val_accuracy', factor = 0.2, patience =  12 )

In [12]:
model.compile(optimizer=Adam(1e-4),
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

In [13]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)        

In [14]:
generator = ImageDataGenerator(rotation_range=90,
                               zoom_range = 0.2,
                               horizontal_flip = True,
                               vertical_flip = True,
                               width_shift_range = 0.1,
                               height_shift_range = 0.1,
                               validation_split = 0.2)

In [15]:
model.fit(generator.flow(x_train,y_train,batch_size = 25),
          validation_data = generator.flow(x_train,y_train,batch_size = 25, subset = 'validation'),
          steps_per_epoch = len(x_train)/25,
          epochs = 100,
          verbose =  2,
          callbacks=[early_stopping, lr_sechudle])



Epoch 1/100
2100/2100 - 36s - loss: 2.0613 - accuracy: 0.3027 - val_loss: 1.5089 - val_accuracy: 0.4535
Epoch 2/100
2100/2100 - 35s - loss: 1.7263 - accuracy: 0.3836 - val_loss: 1.4002 - val_accuracy: 0.5005
Epoch 3/100
2100/2100 - 35s - loss: 1.6255 - accuracy: 0.4143 - val_loss: 1.3459 - val_accuracy: 0.5154
Epoch 4/100
2100/2100 - 36s - loss: 1.5639 - accuracy: 0.4352 - val_loss: 1.3290 - val_accuracy: 0.5278
Epoch 5/100
2100/2100 - 36s - loss: 1.5266 - accuracy: 0.4485 - val_loss: 1.3040 - val_accuracy: 0.5310
Epoch 6/100
2100/2100 - 36s - loss: 1.4941 - accuracy: 0.4618 - val_loss: 1.2838 - val_accuracy: 0.5387
Epoch 7/100
2100/2100 - 36s - loss: 1.4803 - accuracy: 0.4672 - val_loss: 1.2722 - val_accuracy: 0.5488
Epoch 8/100
2100/2100 - 36s - loss: 1.4676 - accuracy: 0.4752 - val_loss: 1.2616 - val_accuracy: 0.5508
Epoch 9/100
2100/2100 - 36s - loss: 1.4544 - accuracy: 0.4771 - val_loss: 1.2637 - val_accuracy: 0.5488
Epoch 10/100
2100/2100 - 36s - loss: 1.4450 - accuracy: 0.4802 -

In [16]:
model.save('model.h5')